<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/single_cls_llama3_2_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torch
!pip install transformers
!pip install peft
!pip install huggingface_hub
!pip install evaluate

In [2]:
from google.colab import userdata
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType  # PEFT 라이브러리 필요
import pandas as pd
from google.colab import drive
from sklearn.metrics import f1_score



### 1. Import data

In [3]:
from google.colab import drive

drive.mount('/content/drive', force_remount=False)
train_path = ('/content/drive/MyDrive/DataSet_new/stackexchange_train_dataset_10000_title.csv')
valid_path = ('/content/drive/MyDrive/DataSet_new/stackexchange_val_dataset_1000_title.csv')
test_path = ('/content/drive/MyDrive/DataSet_new/stackexchange_test_dataset_1000_title.csv')

Mounted at /content/drive


In [4]:
df_train = pd.read_csv(train_path)
df_valid = pd.read_csv(valid_path)
df_test = pd.read_csv(test_path)
#label_names = list(df_train.columns[5:])

In [5]:
df_train.head(1)

,title_content,label,Tags_new,Algorithms,Backend,Data Science,Databases,Dev Tools,Frontend,Mobile,Systems,iOS/macOS
0,The ring formation of ringworm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
num_labels= len(df_train['label'].unique())


### 2. Data Preparation

In [7]:

text_train = list(df_train['title_content'])
label_train = list(df_train['label'])
text_valid = list(df_valid['title_content'])
label_valid = list(df_valid['label'])


In [8]:
# Check the data set to confirm
print(f'Text example 1:\n {text_train[0]} , and its Label:\n {label_train[0]}')


Text example 1:
 The ring formation of ringworm , and its Label:
 0


### 3. Tokenizer and model setting

In [9]:
# 1. Load the tokenizer and model
token =userdata.get('Hugging_Face_llama3.2')
model_path = 'meta-llama/Llama-3.2-1B'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=token)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [10]:
# 2. Tokenizing Dataset

class SingleLabelDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["label"] = torch.tensor(self.labels[idx], dtype=torch.long)  # 단일 클래스 ID로 설정
        return item

    def __len__(self):
        return len(self.labels)

def generate_dataset(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=100, return_tensors="pt")
    dataset = SingleLabelDataset(encodings, labels)
    return dataset

train_dataset = generate_dataset(text_train, label_train)
valid_dataset = generate_dataset(text_valid, label_valid)


In [11]:
# 3. Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # 시퀀스 분류 작업
    r=8,  # 저차원 행렬의 랭크
    lora_alpha=16,  # 스케일링 인자
    lora_dropout=0.1  # 드롭아웃 비율
)

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, use_auth_token=token)
model.config.pad_token_id = tokenizer.pad_token_id


# 3. Apply LoRA to the model
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

import wandb

# wandb 초기화 (API 키가 환경 변수에 설정되어 있어야 함)
wandb.init(project="Single label classification", entity="ai_expert", name="Llama3_2_1b_re2")
# wandb.init(project="huggingface") # Uncomment to use wandb


accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "macro_precision": precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "micro_precision": precision_metric.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall_metric.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1_metric.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "weighted_precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }
    return metrics

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/single_label",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=5,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  report_to="wandb",  # wandb를 사용하도록 설정
                                  logging_strategy="epoch"
                                  )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset = valid_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()

wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1,Weighted Precision,Weighted Recall,Weighted F1
1,0.227700,0.121590,0.977000,0.977088,0.977000,0.977009,0.977000,0.977000,0.977000,0.977088,0.977000,0.977009
2,0.062700,0.136253,0.980000,0.980187,0.980000,0.980026,0.980000,0.980000,0.980000,0.980187,0.980000,0.980026
3,0.035500,0.157716,0.980000,0.980188,0.980000,0.980011,0.980000,0.980000,0.980000,0.980188,0.980000,0.980011
4,0.016400,0.157717,0.981000,0.981126,0.981000,0.981017,0.981000,0.981000,0.981000,0.981126,0.981000,0.981017
5,0.007800,0.160759,0.981000,0.981126,0.981000,0.981017,0.981000,0.981000,0.981000,0.981126,0.981000,0.981017


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6730baf0-6cde841907264ed33429f9ab;a16cf306-6764-48a9-9d91-8c6094e78706)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B - will assume that th

eval/accuracy,▁▆▆██
eval/loss,▁▄▇▇█
eval/macro_f1,▁▆▆██
eval/macro_precision,▁▆▆██
eval/macro_recall,▁▆▆██
eval/micro_f1,▁▆▆██
eval/micro_precision,▁▆▆██
eval/micro_recall,▁▆▆██
eval/runtime,█▁▂▁▃
eval/samples_per_second,▁█▇█▆
eval/steps_per_second,▁█▇█▆
